In [ ]:
#| default_exp core

# core

> Tiny helper to authenticate Claudette to Vertex AI

In [ ]:
#| export
import json, os
from pathlib import Path
from fastcore.utils import *

from anthropic import AnthropicVertex, AsyncAnthropicVertex
from claudette import Client, AsyncClient


## Optional: Setting up to authenticate with Vertex AI

If you already have the vertexauth "superkey.json", then you may skip this section and proceed to _Authenticating to Claudette_.

Otherwise, this is example code to create a Google Cloud (GC) Service Account, and from there create a GC Service Account Key file, and from there create a vertexauth superkey.json

### 1. Creating a GC service account

In [ ]:
# !pip install -q google-cloud-service-usage google-cloud-iam google-cloud-resource-manager

1. We'll need to use the `google.cloud.iam_admin_v1` client library
2. The documentation notes that we'll need:
   - project_id: The Google Cloud project ID
   - account: The service account ID or email

In [ ]:
#|eval: false

import google.auth
from google.cloud import iam_admin_v1
from google.cloud.iam_admin_v1 import types
from google.cloud import resourcemanager_v3
from google.iam.v1 import policy_pb2

We can work with Google's IAM API by using a client, assuming you have first run `gcloud auth application-default login`.

In [ ]:
project_id = 'jph001'

In [ ]:
#|eval: false

cli = iam_admin_v1.IAMClient()
credentials, project_id = google.auth.default()
project = f"projects/{project_id}"
project

'projects/jph001'

We will need a "service account" with the appropriate permissions. You can check your account list like so:

In [ ]:
#|eval: false

accounts = cli.list_service_accounts(name = project)
# accounts

...and here is how to create an account:

In [ ]:
#|eval: false

account_id="aiservice2"
display_name="Vertex AI Service Account 2"
description="Access Vertex AI"

In [ ]:
#|eval: false

svc = dict(display_name=display_name, description=description)
account = cli.create_service_account(name=project, account_id=account_id, service_account=svc)
# account

In [ ]:
#|eval: false

polcli = resourcemanager_v3.ProjectsClient()
policy = polcli.get_iam_policy(resource=project)

In [ ]:
#|eval: false

member = f"serviceAccount:{account.email}"
roles = [ "roles/aiplatform.user", "roles/servicemanagement.quotaViewer", "roles/servicemanagement.quotaAdmin" ]

In [ ]:
#|eval: false

for role in roles:
    binding = policy_pb2.Binding()
    binding.role = role
    binding.members.append(member)
    policy.bindings.append(binding)
    
polres = polcli.set_iam_policy(request={"resource": project, "policy": policy})

(If you later wanted to delete a service account, you could the following:)

In [ ]:
# cli.delete_service_account(name=f"projects/{project_id}/serviceAccounts/aiservice@jph001.iam.gserviceaccount.com")

### 2. Creating a GC Service Account Key File (SAKF)

In [ ]:
#|eval: false

key = cli.create_service_account_key(name = f"projects/{project_id}/serviceAccounts/{account.email}")

In [ ]:
#|eval: false

keyd = json.loads(key.private_key_data.decode())
keyb = json.dumps(keyd).encode()

In [ ]:
#|eval: false

path = Path('service-account-key.json')
path.write_bytes(keyb)

2329

### 3. Creating a vertexauth "superkey"

A vertexauth "superkey" is merely a SAKF file with a "region" key/value pair added so that it can be the only resource needed in order to use this library.

If a colleague already gave you a superkey, save it in `~/.config/vertexauth/default/superkey.json`, and skip to the next section _Authenticating to Claudette_. But if you only have a SAKF JSON file, as created above or downloaded from the Google Cloud web UI, then you create and save a superkey as follows:

In [ ]:
#| export

SUPERKEY_DEFAULT_PATH = Path.home() / ".config" / "vertexauth" / "default" / "superkey.json"

In [ ]:
def save_superkey_file(SAKF_path, region) -> Path:
    d = json.loads(Path(SAKF_path).read_text())
    d["region"] = region
    SUPERKEY_DEFAULT_PATH.parent.mkdir(parents=True,exist_ok=True)
    SUPERKEY_DEFAULT_PATH.write_text(json.dumps(d))
    SUPERKEY_DEFAULT_PATH.chmod(0o600)
    return SUPERKEY_DEFAULT_PATH

Save a superkey based on the path of the service account key we just created above and saved in `path`.

In [ ]:
#|eval: false

save_superkey_file(path,'us-east5')

Path('/Users/alexis/.config/vertexauth/default/superkey.json')

## Authenticating to Claudette

Once you have a superkey file (i.e., a JSON SAKF plus a region key/value pair), then you can use these functions to create a claudette client or an Anthropic client.

In [ ]:
#| export

def get_anthropic_client(asink=False,anthropic_kwargs=None):
    d = json.loads(SUPERKEY_DEFAULT_PATH.read_text())
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = str(SUPERKEY_DEFAULT_PATH)
    os.environ["GOOGLE_CLOUD_PROJECT"]           = d['project_id']
    anthropic_kwargs = anthropic_kwargs or {}
    AV = AnthropicVertex if not asink else AsyncAnthropicVertex
    return AV(region=d['region'],project_id=d['project_id'], **anthropic_kwargs)    

def get_claudette_client(vertex_model='claude-3-5-sonnet-v2@20241022', 
                         asink=False, anthropic_kwargs=None, cache=False):
    vertex_cli = get_anthropic_client(asink, anthropic_kwargs)
    if asink: return AsyncClient(vertex_model, vertex_cli, cache=cache)
    else: return Client(vertex_model, vertex_cli, cache=cache)

## Using claudette

In [ ]:
#|eval: false

cl = get_claudette_client()

In [ ]:
#|eval: false

cl('hi')

Hello! How can I help you today?

<details>

- id: `msg_vrtx_013ZqSR5aJtQ1W6XkgsQ1FHP`
- content: `[{'text': 'Hello! How can I help you today?', 'type': 'text'}]`
- model: `claude-3-5-sonnet-v2-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'input_tokens': 8, 'output_tokens': 12}`

</details>

## Todo: quota management

In [ ]:
#|eval: false

quota_docs = read_gist('https://gist.github.com/jph00/943c51623abfe0deae65cfad2d821169')
svcuse_docs = read_gist('https://gist.github.com/jph00/042580724e98ae0cce2db50de92abd1b')

In [ ]:
#|eval: false

from google.cloud import service_usage_v1

Yes I see some options we could use -- do you want me to outline them now?

In [ ]:
#|eval: false

scli = service_usage_v1.ServiceUsageClient()
services = scli.list_services(request={"parent": project, "filter":"state:ENABLED"})

## export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()